## Imports

In [89]:
import random
from typing import List, Dict, Tuple

## Clases para el algoritmo genetico

In [90]:
class Gene:
    def __init__(self, curso: str, profesor: str, salon: str, dia: str, hora: str, programa: str, creditos: int, semestre: int, estudiante: int):
        self.curso = curso
        self.profesor = profesor
        self.salon = salon
        self.dia = dia
        self.hora = hora
        self.programa = programa
        self.creditos = creditos
        self.semestre = semestre
        self.estudiante = estudiante

class Chromosome:
    def __init__(self, genes: List[Gene]):
        self.genes = genes
        self.fitness = 0


In [91]:
class GeneticAlgorithm:
    def __init__(self, cursos: List[Dict], salones: List[str], dias: List[str], horas: List[str], 
                 population_size: int, mutation_rate: float, crossover_rate: float,
                 salon_props: Dict[str, Dict], fitness_settings: Dict):
        self.cursos = cursos
        self.salones = salones
        self.dias = dias
        self.horas = horas
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.salon_props = salon_props
        self.fitness_settings = fitness_settings
        self.population = []

    def fitness_function(self, chromosome: Chromosome) -> float:
        penalty = 0

        for gene in chromosome.genes:
            # Check salon capacidad
            salon_capacidad = self.salon_props.get(gene.salon, {}).get('capacidad', 0)
            if salon_capacidad < gene.estudiante:
                penalty += self.fitness_settings['salonOverCapacidad']['penalty']
                
                # Check hora constraints
                #hora_start = int(gene.hora.split(' - ')[0].split('.')[0])
                #if (hora_start > 3 and hora_start < 6) or (hora_start < 4 and (hora_start + gene.creditos - 1) > 3):
                 #   penalty += 1
                
                #if (gene.dia == 'jueves' and hora_start > 2 and hora_start < 6) or \
                #(gene.dia == 'jueves' and hora_start < 3 and (hora_start + gene.creditos - 1) > 2):
                 #   penalty += 1
        
        # Check for conflicts
        for i, gene1 in enumerate(chromosome.genes):
            for j, gene2 in enumerate(chromosome.genes[i+1:]):
                if gene1.dia == gene2.dia and gene1.hora == gene2.hora:
                    if gene1.salon == gene2.salon:
                        penalty += self.fitness_settings['sameSalonSameHora']['penalty']
                    if gene1.profesor == gene2.profesor:
                        penalty += self.fitness_settings['sameProfesorSameHora']['penalty']
                    if gene1.programa == gene2.programa and gene1.semestre == gene2.semestre:
                        penalty += self.fitness_settings['sameProgramaSameSemesterSameHora']['penalty']
        
        return 1 / (1 + penalty)

    def evolve(self, generations: int):
            self.initialize_population()
            
            for _ in range(generations):
                new_population = []
                
                while len(new_population) < self.population_size:
                    parent1 = self.tournament_selection(3)
                    parent2 = self.tournament_selection(3)
                    
                    if random.random() < self.crossover_rate:
                        child1, child2 = self.uniform_crossover(parent1, parent2)
                    else:
                        child1 = Chromosome([Gene(g.curso, g.profesor, g.salon, g.dia, g.hora,
                                                g.programa, g.creditos, g.semestre, g.estudiante) for g in parent1.genes])
                        child2 = Chromosome([Gene(g.curso, g.profesor, g.salon, g.dia, g.hora,
                                                g.programa, g.creditos, g.semestre, g.estudiante) for g in parent2.genes])
                    
                    self.mutate(child1)
                    self.mutate(child2)
                    
                    for child in [child1, child2]:
                        child.fitness = self.fitness_function(child)
                    
                    new_population.extend([child1, child2])
                
                self.population = sorted(new_population, key=lambda x: x.fitness, reverse=True)[:self.population_size]
            
            best_solution = max(self.population, key=lambda chromo: chromo.fitness)
            top5 = sorted(self.population, key=lambda chromo: chromo.fitness, reverse=True)[:5]
            return best_solution, top5
    
    def initialize_population(self):
        for _ in range(self.population_size):
            chromosome = Chromosome([
                Gene(curso['curso'], curso['profesor'], 
                     random.choice(self.salones), 
                     random.choice(self.dias), 
                     random.choice(self.horas),
                     curso['programa'], curso['creditos'], curso['semestre'], curso['estudiante'])
                for curso in self.cursos
            ])
            self.population.append(chromosome)

    def tournament_selection(self, tournament_size: int) -> Chromosome:
        tournament = random.sample(self.population, tournament_size)
        return max(tournament, key=lambda chromo: chromo.fitness)
    
    def uniform_crossover(self, parent1: Chromosome, parent2: Chromosome) -> Tuple[Chromosome, Chromosome]:
        child1_genes = []
        child2_genes = []
        for gene1, gene2 in zip(parent1.genes, parent2.genes):
            if random.random() < 0.5:
                child1_genes.append(Gene(gene1.curso, gene1.profesor, gene1.salon, gene1.dia, gene1.hora,
                                         gene1.programa, gene1.creditos, gene1.semestre, gene1.estudiante))
                child2_genes.append(Gene(gene2.curso, gene2.profesor, gene2.salon, gene2.dia, gene2.hora,
                                         gene2.programa, gene2.creditos, gene2.semestre, gene2.estudiante))
            else:
                child1_genes.append(Gene(gene2.curso, gene2.profesor, gene2.salon, gene2.dia, gene2.hora,
                                         gene2.programa, gene2.creditos, gene2.semestre, gene2.estudiante))
                child2_genes.append(Gene(gene1.curso, gene1.profesor, gene1.salon, gene1.dia, gene1.hora,
                                         gene1.programa, gene1.creditos, gene1.semestre, gene1.estudiante))
        return Chromosome(child1_genes), Chromosome(child2_genes)
    
    def mutate(self, chromosome: Chromosome):
        for gene in chromosome.genes:
            if random.random() < self.mutation_rate:
                gene.salon = random.choice(self.salones)
                gene.dia = random.choice(self.dias)
                gene.hora = random.choice(self.horas)

### Input

In [92]:
dias = ["Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado"]
horas = ["08.00 - 08.50", "09.00 - 09.50", "10.00 - 10.50", "11.00 - 11.50", "19.00 - 19.50"]
salones = ["R.1&2", "R.3", "R.4", "R.5", "R.6", "R.7", "R.8", "R.9&10"]
salon_props = {
    "R.1&2": {"owner": ["Derecho", "Derecho (AM)", "Administracion", "Contabilidad"], "capacidad": 150},
    "R.3": {"owner": ["all"], "capacidad": 40},
    "R.4": {"owner": ["all"], "capacidad": 40},
    "R.5": {"owner": ["all"], "capacidad": 40},
    "R.6": {"owner": ["all"], "capacidad": 40},
    "R.7": {"owner": ["all"], "capacidad": 40},
    "R.8": {"owner": ["all"], "capacidad": 40},
    "R.9&10": {"owner": ["all"], "capacidad": 80}
}

cursos = [
    {"programa": "Administracion", "curso": "Introduccion a la microeconomía", "estudiante": 30,
     "profesor": "Duvan", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Administracion", "curso": "Introduccion a la macroeconomía", "estudiante": 30,
    "profesor": "Duvan", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Administracion", "curso": "Introduccion a la contabilidad", "estudiante": 30,
    "profesor": "Duvan", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Administracion", "curso": "Introduccion a la administracion", "estudiante": 30,
    "profesor": "Duvan", "creditos": 3, "semestre": 1, "required": True},
]

ga_params = {
    "nPopulations": 1000,
    "npops": 10,
    "nSelection": 3,
    "pCrossover": 0.85,
    "pMutation": 0.14,
    "fitnessThresshold": 0.8,
    "nSolution": 1,
    "stoppingCondition": {"nFitnessNoChange": 1000, "fitnessMax": 1}
}

fitness_settings = {
    "sameProfesorSameHora": {"enable": True, "penalty": 1},
    "sameProgramaSameSemesterSameHora": {"enable": True, "penalty": 1},
    "sameSalonSameHora": {"enable": True, "penalty": 1},
    "horaOver": {"enable": True, "penalty": 1},
    "salonOverCapacidad": {"enable": True, "penalty": 1},
    "salonUsedByOthers": {"enable": False, "penalty": 0.01},
    "sameProfesorSameDia": {"enable": False, "penalty": 0.001},
    "sameProfesorHasSequence": {"enable": False, "penalty": 0.001},
    "sameProgramaSameSemesterSameDia": {"enable": False, "penalty": 0.001},
    "sameProgramaSameSemesterHasSequence": {"enable": False, "penalty": 0.001}
}

## Solucionar

In [93]:
ga = GeneticAlgorithm(
    cursos=cursos,
    salones=salones,
    dias=dias,
    horas=horas,
    population_size=ga_params["nPopulations"],
    mutation_rate=ga_params["pMutation"],
    crossover_rate=ga_params["pCrossover"],
    salon_props=salon_props,
    fitness_settings=fitness_settings
)

best_solution, top5 = ga.evolve(generations=ga_params["stoppingCondition"]["nFitnessNoChange"])

# Output
print("\nBest Solution:")
print(f"Fitness: {best_solution.fitness:.4f}")
print("Schedule:")
for i, gene in enumerate(best_solution.genes, 1):
    print(f"{i}. curso: {gene.curso}")
    print(f"   profesor: {gene.profesor}")
    print(f"   salon: {gene.salon}")
    print(f"   dia: {gene.dia}")
    print(f"   hora: {gene.hora}")
    print(f"   programa: {gene.programa}")
    print(f"   creditos: {gene.creditos}")
    print(f"   Semester: {gene.semestre}")
    print(f"   estudiantes: {gene.estudiante}")
    print()

for i, solution in enumerate(top5, 1):
    print(f"\nTop {i} Solution:")
    print(f"Fitness: {solution.fitness:.4f}")
    print("Schedule:")
    for i, gene in enumerate(solution.genes, 1):
        print(f"{i}. curso: {gene.curso}")
        print(f"   profesor: {gene.profesor}")
        print(f"   salon: {gene.salon}")
        print(f"   dia: {gene.dia}")
        print(f"   hora: {gene.hora}")
        print(f"   programa: {gene.programa}")
        print(f"   creditos: {gene.creditos}")
        print(f"   Semester: {gene.semestre}")
        print(f"   estudiantes: {gene.estudiante}")
        print()


Best Solution:
Fitness: 1.0000
Schedule:
1. curso: Introduccion a la microeconomía
   profesor: Duvan
   salon: R.1&2
   dia: Lunes
   hora: 10.00 - 10.50
   programa: Administracion
   creditos: 3
   Semester: 1
   estudiantes: 30

2. curso: Introduccion a la macroeconomía
   profesor: Duvan
   salon: R.3
   dia: Martes
   hora: 08.00 - 08.50
   programa: Administracion
   creditos: 3
   Semester: 1
   estudiantes: 30

3. curso: Introduccion a la contabilidad
   profesor: Duvan
   salon: R.4
   dia: Lunes
   hora: 09.00 - 09.50
   programa: Administracion
   creditos: 3
   Semester: 1
   estudiantes: 30

4. curso: Introduccion a la administracion
   profesor: Duvan
   salon: R.3
   dia: Sabado
   hora: 11.00 - 11.50
   programa: Administracion
   creditos: 3
   Semester: 1
   estudiantes: 30


Top 1 Solution:
Fitness: 1.0000
Schedule:
1. curso: Introduccion a la microeconomía
   profesor: Duvan
   salon: R.1&2
   dia: Lunes
   hora: 10.00 - 10.50
   programa: Administracion
   credit